<a href="https://colab.research.google.com/github/GitZy6666/ZyRep/blob/main/Re_FeatureExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install heartpy

In [ ]:
import numpy as np
import pandas as pd
import heartpy as hp
from scipy.signal import resample


import warnings
warnings.filterwarnings('ignore')

In [ ]:
event = pd.read_csv('anotasiEvent.csv')
event = event.to_numpy()

In [ ]:
dataset = pd.DataFrame([])
dataset1 = pd.DataFrame([])

for i in range(0, len(event)):

  subject = event[i,0]
  from_idx = event[i,1]
  upto_idx = event[i,2]
  label = event[i, 3]

  data = pd.read_csv('Patient'+str(subject)+'.csv')

  ECG = data['II']
  PPG = data['Pleth']
  time = data['Time']

  ECG = ECG[from_idx:upto_idx]
  PPG = PPG[from_idx:upto_idx]
  time = time[from_idx:upto_idx]

  ECG = ECG.to_numpy()
  PPG = PPG.to_numpy()
  time = time.to_numpy()

  sample_rate = hp.get_samplerate_datetime(time, timeformat = '%H:%M:%S.%f')

  denoise = hp.filter_signal(PPG, 
                                  cutoff = [0.8, 2.5], 
                                  filtertype = 'bandpass',
                                  sample_rate = sample_rate, 
                                  order = 3,
                                  return_top = False)

  resampled = resample(denoise, len(denoise) * 10)
  new_sample_rate = sample_rate * 10

  wd, m = hp.process(resampled, sample_rate = new_sample_rate, high_precision=True, clean_rr=False)

  dataset1 = pd.DataFrame([m])
  dataset1['desc'] = str(subject)+'.'+str(from_idx)+'.'+str(upto_idx)
  dataset1['label'] = label
  dataset = dataset.append(dataset1, ignore_index=True)

In [ ]:
dataset.sample(10)

,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate,desc,label
66,100.614733,596.334135,62.577688,34.868980,67.368252,0.800000,0.600000,59.923077,47.527407,72.532917,10830.017051,0.655253,0.419551,6.1700.2220,PVC
122,77.175408,777.449728,64.568367,72.419292,86.750162,0.375000,0.250000,24.937600,61.110519,51.704506,9926.456419,1.181919,0.308769,10.5000.6000,Normal
58,97.733311,613.915556,135.371349,123.911217,224.986830,0.933333,0.866667,74.620000,158.206036,100.282883,49842.490279,1.577598,0.543577,5.12000.13000,PVC
35,82.435044,727.845789,111.852585,86.005767,151.965464,0.971429,0.714286,71.760000,107.450378,120.986508,40840.849000,0.888119,0.180923,4.1000.2500,AF
111,74.183910,808.800727,83.832095,80.987964,101.026065,0.750000,0.300000,28.924600,69.235069,66.819983,14533.906572,1.036143,0.225023,9.10000.11000,Normal
109,78.178144,767.477928,59.865073,60.893792,71.661140,0.583333,0.083333,23.938800,49.894035,47.001723,7367.366512,1.061536,0.052151,9.8000.9000,Normal
95,123.962988,484.015439,93.821837,94.381344,136.790860,0.789474,0.657895,51.856260,95.243596,80.584468,24112.208894,1.181910,0.443350,7.6400.7630,PVC
116,77.099497,778.215192,78.257765,72.634355,87.431010,0.652174,0.173913,40.861400,61.579274,84.679600,16381.860880,0.727203,0.107152,9.16000.17000,Normal
98,116.170026,516.484348,108.576895,99.978725,156.462754,0.846154,0.692308,69.905838,110.604419,107.280437,37277.165874,1.030984,0.645797,7.11060.13030,PVC
129,79.005597,759.439868,37.249231,38.964734,59.833537,0.833333,0.291667,21.954900,42.271904,31.637519,4201.497450,1.336132,0.579710,10.14000.15000,Normal


In [ ]:
dataset.to_csv('datasetFx1.csv', index=False)